In [ ]:
import duckdb
import geopandas as gpd
import pandas as pd
from h3pandas.util.shapely import polyfill
import pyarrow.dataset as ds


In [ ]:

wkt = "POLYGON ((2.540588 51.094898, 2.381287 51.272226, 3.073425 51.554874, 3.372803 51.361492, 2.540588 51.094898))"
gs = gpd.GeoSeries.from_wkt([wkt])
h3 = list(polyfill(gs[0], 7, geo_json=True))
poly = pd.DataFrame({"h3": h3})
poly

In [ ]:
con = duckdb.connect()
con.register("poly", poly)

dataset = ds.dataset("../h3_7/", format="parquet")
con.register("dataset", dataset)

df = con.execute("""
	select
        species,
        sum(records) as records,
        min(min_year) as min_year,
        max(max_year) as max_year,
        max(source_obis) as source_obis,
        max(source_gbif) as source_gbif
    from dataset
    inner join poly on poly.h3 = dataset.h3_07
    group by species
""").fetchdf()
df